Markov Chain Monte Carlo (MCMC) methods are a fascinating group of algorithms that allow to sample from distributions that are often not tractable analytically. In this notebook we look at a hands-on overview of MCMC - we will see why these methods are useful and apply MCMC in the context of a simple Bernoulli trials problem which lends itself to a Bayesian setup. Since the binomial distribution has a conjugate prior in the Beta distribution, our particular problem can be solved analytically, allowing us to verify our simulation results obtained via MCMC.

### Problem Outline

Let's look at the following problem: We have a biased coin that gives us "Heads" with probability $\theta$. $\theta$ is a probability, so surely $ \theta \in [0, 1] $, but beyond that we don't know much. Adopting a Bayesian viewpoint, we want to treat $\theta$ as a random variable and describe it in terms of a distribution. Initially however, we don't have any particular view, so our prior is flat and a simple uniform distribution  
$$ \theta \sim U[0, 1], $$

that is we believe that all values between 0 and 1 are equally likely and $\mathbb{E}[\theta] = 0.5$.


But we can now throw the coin and update our view as to what the coin's probability $\theta$ is likely to be. Each throw will give us a new data point that we will use to update our prior, and according to Bayes' rule we have:
$$ p(\theta | Data) = \frac{p(Data | \theta) p(\theta)}{p(Data)}$$

where p($\theta$) is our uniform prior.

The challenge in these problems is typically the normalisation factor that turns out to be intractable, here $p(Data)$. The integral that it involves is typically difficult or impossible to solve, not allowing us to compute the pdf of the resulting conditional distribution. (In this particular example it is solvable, more on that later).

We can write:

$$ p(\theta | Data) = \frac{p(Data | \theta) p(\theta)}{p(Data)}$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import seaborn as sns


In [ ]:
data = [7.]

In [ ]:
posterior = tfd.JointDistributionSequential([
    ## prior assumption over the mus:
    tfd.Uniform(low=[0.0], high=[1.0]), lambda theta:
    tfd.Independent(tfd.Binomial(total_count=[10.], probs=theta), reinterpreted_batch_ndims=1)
])

In [ ]:
def post_log_prob(theta):
    return tf.reshape(posterior.log_prob(([theta], data)), [])

In [ ]:
num_results, num_burnin_steps = 40000, 10000

In [ ]:
@tf.function(autograph=False)
def do_sampling():
  return tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=[tf.constant([0.5])],
      kernel=adaptive_kernel)

hmc_kernel=tfp.mcmc.HamiltonianMonteCarlo(
          target_log_prob_fn=post_log_prob,
          step_size=0.4,
          num_leapfrog_steps=2)

adaptive_kernel = tfp.mcmc.SimpleStepSizeAdaptation(
  hmc_kernel,
  num_adaptation_steps=int(.8 * num_burnin_steps),
  target_accept_prob=np.float32(.65))

In [ ]:
states, kernel_results = do_sampling()

In [2]:
def uniform(a,b):
    return 1/(b-a)

def binomial(n, k, p):
    bc = np.math.factorial(n)/np.math.factorial(k)/np.math.factorial(n-k)
    return bc *(p**k) * ((1-p)**(n-k))

k throws are heads:

In [6]:
binomial(1, 1, theta)

0.0439453125

In [3]:
N = 100
s = 10
x = 0
p = binomial(1,1,x)
samples = []

In [4]:
for i in range(N):
    # proposal distribution is normal:
    xn = x + np.random.normal(size=1)
    pn = binomial(1, 1, xn)
    
    if pn >= p:
        p = pn
        x = xn
    else:
        u = np.random.rand()
        if u < pn/p:
            p = pn
            x = xn
    if i % s == 0:
        samples.append(x)




C:\Users\Hendrik\AppData\Local\Temp/ipykernel_417164/3930575857.py:23: RuntimeWarning: divide by zero encountered in true_divide
  if u < pn/p:


In [ ]:
samples = np.array(samples)

plt.scatter(samples, np.zeros_like(samples), s=10)